<a href="https://colab.research.google.com/github/hikaru122700/kaggle-private/blob/SMBC/017_catboost_after_act_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pygeohash
!pip install catboost
!pip install lightgbm
!pip install xgboost
!pip install joblib
!pip install optuna

  Preparing metadata (setup.py) ... done
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6153 sha256=3160f7824e486b75d1b320d16f8c9c8dd0bb40ef8c8622b10d7839b88faf1509
  Stored in directory: /root/.cache/pip/wheels/28/ec/b6/beadf7295a623f528507691fb0d471b50d064ae9bbad420b8f
Successfully built pygeohash
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00


In [4]:

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
import math

import pygeohash as pgh

from concurrent.futures import ThreadPoolExecutor

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder

import gc
gc.collect()
import optuna


import os

In [21]:
path  = "/content/drive/My Drive/signate/SMBC Group GREEN×DATA Challenge 2024/"
# 予測モデルを訓練するためのデータセット
train_df = pd.read_csv(path+'train_all_df.csv', index_col=0)
# train_df.drop("GHG_Direct_Emissions_14_in_metric_tons", axis=0)
# 予測モデルに推論（予測)させるデータセット
test_df = pd.read_csv(path+'test_all_df.csv', index_col=0)

In [23]:
train_df.drop("GHG_Direct_Emissions_14_in_metric_tons", axis=1, inplace=True)

In [24]:
train_df

,TRI_Air_Emissions_11_in_lbs,TRI_Air_Emissions_12_in_lbs,TRI_Air_Emissions_13_in_lbs,PrimaryNAICS,GHG_Direct_Emissions_10_in_metric_tons,GHG_Direct_Emissions_11_in_metric_tons,GHG_Direct_Emissions_12_in_metric_tons,GHG_Direct_Emissions_13_in_metric_tons,Latitude,Longitude,TRI_Air_Emissions_YoY_Change_11,TRI_Air_Emissions_YoY_Change_12,TRI_Air_Emissions_YoY_Change_13,TRI_Air_Emissions_Growth_Rate_11,TRI_Air_Emissions_Growth_Rate_12,TRI_Air_Emissions_Growth_Rate_13,Economic_Sector,num_missing,oof_predictions,GHG_Direct_Emissions_14_in_metric_tons.1
TRI_Air_Emissions_10_in_lbs,,,,,,,,,,,,,,,,,,,,
31566.709644,26644.986107,23410.379903,31809.857564,118,64816.958901,36588.744606,3.790794e+04,45598.125851,40.141389,-87.581111,-4921.723537,-3234.606204,8399.477661,-0.155915,-0.121396,0.358793,8,0,3.868125e+04,5.297314e+04
NaN,NaN,NaN,NaN,221,55507.543666,72387.334115,5.822520e+04,76376.547318,38.475305,-81.278957,NaN,NaN,NaN,NaN,NaN,NaN,14,1,6.421108e+04,5.591007e+04
NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,39.299820,-80.857170,NaN,NaN,NaN,NaN,NaN,NaN,9,1,1.092220e+05,5.567954e+04
NaN,NaN,NaN,NaN,21,54811.222708,69339.923002,6.364734e+04,53799.011225,35.490363,-119.042957,NaN,NaN,NaN,NaN,NaN,NaN,15,1,5.115422e+04,6.141190e+04
29553.796627,28337.832145,30840.825454,25153.901905,21,81812.306362,53823.561587,7.739116e+04,17662.966241,41.532802,-81.636448,-1215.964481,2502.993309,-5686.923550,-0.041144,0.088327,-0.184396,15,0,5.271591e+04,4.310047e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,3,54978.841280,56463.765036,3.522786e+06,76642.640670,32.523334,-97.102556,NaN,NaN,NaN,NaN,NaN,NaN,9,1,2.220249e+05,2.451197e+06
NaN,NaN,NaN,NaN,3,208375.435367,57066.624445,4.858030e+04,26892.366779,42.684900,-109.780000,NaN,NaN,NaN,NaN,NaN,NaN,9,1,5.036959e+04,3.573974e+04
NaN,NaN,NaN,NaN,244,563756.090695,598881.479485,1.665103e+06,543206.073134,35.484260,-97.498570,NaN,NaN,NaN,NaN,NaN,NaN,5,1,1.211018e+06,9.783592e+05


In [25]:
train = train_df.iloc[:, :-1].values
test = test_df.values
target = train_df.iloc[:, -1].values

In [26]:
train_df.iloc[:, :-1]

,TRI_Air_Emissions_11_in_lbs,TRI_Air_Emissions_12_in_lbs,TRI_Air_Emissions_13_in_lbs,PrimaryNAICS,GHG_Direct_Emissions_10_in_metric_tons,GHG_Direct_Emissions_11_in_metric_tons,GHG_Direct_Emissions_12_in_metric_tons,GHG_Direct_Emissions_13_in_metric_tons,Latitude,Longitude,TRI_Air_Emissions_YoY_Change_11,TRI_Air_Emissions_YoY_Change_12,TRI_Air_Emissions_YoY_Change_13,TRI_Air_Emissions_Growth_Rate_11,TRI_Air_Emissions_Growth_Rate_12,TRI_Air_Emissions_Growth_Rate_13,Economic_Sector,num_missing,oof_predictions
TRI_Air_Emissions_10_in_lbs,,,,,,,,,,,,,,,,,,,
31566.709644,26644.986107,23410.379903,31809.857564,118,64816.958901,36588.744606,3.790794e+04,45598.125851,40.141389,-87.581111,-4921.723537,-3234.606204,8399.477661,-0.155915,-0.121396,0.358793,8,0,3.868125e+04
NaN,NaN,NaN,NaN,221,55507.543666,72387.334115,5.822520e+04,76376.547318,38.475305,-81.278957,NaN,NaN,NaN,NaN,NaN,NaN,14,1,6.421108e+04
NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,39.299820,-80.857170,NaN,NaN,NaN,NaN,NaN,NaN,9,1,1.092220e+05
NaN,NaN,NaN,NaN,21,54811.222708,69339.923002,6.364734e+04,53799.011225,35.490363,-119.042957,NaN,NaN,NaN,NaN,NaN,NaN,15,1,5.115422e+04
29553.796627,28337.832145,30840.825454,25153.901905,21,81812.306362,53823.561587,7.739116e+04,17662.966241,41.532802,-81.636448,-1215.964481,2502.993309,-5686.923550,-0.041144,0.088327,-0.184396,15,0,5.271591e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,3,54978.841280,56463.765036,3.522786e+06,76642.640670,32.523334,-97.102556,NaN,NaN,NaN,NaN,NaN,NaN,9,1,2.220249e+05
NaN,NaN,NaN,NaN,3,208375.435367,57066.624445,4.858030e+04,26892.366779,42.684900,-109.780000,NaN,NaN,NaN,NaN,NaN,NaN,9,1,5.036959e+04
NaN,NaN,NaN,NaN,244,563756.090695,598881.479485,1.665103e+06,543206.073134,35.484260,-97.498570,NaN,NaN,NaN,NaN,NaN,NaN,5,1,1.211018e+06


In [27]:

# Step 1: Optuna の Objective 関数
def objective(trial, X, y, num_folds):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 10),
        "learning_rate":trial.suggest_float('learning_rate', 0.001, 0.1),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        "random_seed": 13,
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 20),
    }



    kf = KFold(n_splits=num_folds, shuffle=True, random_state=13)
    oof_preds = np.zeros(len(X))

    for train_idx, valid_idx in kf.split(X, y):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = CatBoostRegressor(**params)
        model.fit(X_train, np.log1p(y_train), eval_set=(X_valid, np.log1p(y_valid)), early_stopping_rounds=50, verbose=False)

        preds = np.expm1(model.predict(X_valid))
        rmsle = np.sqrt(mean_squared_log_error(y_valid, preds))
        oof_preds[valid_idx] = preds

    final_rmsle = np.sqrt(mean_squared_log_error(y, oof_preds))
    return final_rmsle


In [28]:

# Step 2: Optuna を用いたハイパーパラメータチューニング
def tune_catboost(X, y, num_folds=5, n_trials=200):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X, y, num_folds), n_trials=n_trials)
    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
    return trial.params

In [29]:

# Step 3: K-Fold Cross-Validation を用いたモデル訓練と予測
def get_models_trained(train, test, target, num_folds, catboost_params):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=13)

    oof_predictions = np.zeros(len(train))
    test_predictions = np.zeros(len(test))

    for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
        print(f"Starting Fold {fold + 1}")
        X_train, X_valid = train[train_index], train[valid_index]
        y_train, y_valid = target[train_index], target[valid_index]

        # CatBoostRegressor のインスタンス作成
        # model = X(**catboost_params, random_state=13, verbose=0)
        model = CatBoostRegressor(**catboost_params, random_state=13, verbose=0)

        # モデルの訓練
        model.fit(X_train, np.log1p(y_train), eval_set=(X_valid, np.log1p(y_valid)), early_stopping_rounds=50, verbose=False)

        # バリデーションデータに対する予測
        valid_preds = np.expm1(model.predict(X_valid))
        rmsle = np.sqrt(mean_squared_log_error(y_valid, valid_preds))
        print(f"Fold {fold + 1} RMSLE = {rmsle}")

        # OOF 予測の保存
        oof_predictions[valid_index] = valid_preds

        # テストデータに対する予測の蓄積
        test_preds = np.expm1(model.predict(test))
        test_predictions += test_preds / num_folds

        # メモリの解放
        del X_train, X_valid, y_train, y_valid, model
        gc.collect()

        print('---------------\n')

    # 最終的な OOF RMSLE の計算
    final_RMSLE = np.sqrt(mean_squared_log_error(target, oof_predictions))
    print(f"OOF RMSLE = {final_RMSLE}")

    return oof_predictions, test_predictions


In [30]:
    # Step 1: ハイパーパラメータの最適化
    best_params = tune_catboost(train, target, num_folds=15, n_trials=200)



[I 2024-11-22 04:28:02,886] A new study created in memory with name: no-name-44e036bb-bae4-4851-8b99-1103b61e91bd
[I 2024-11-22 04:28:12,295] Trial 0 finished with value: 0.7546504932949725 and parameters: {'iterations': 499, 'depth': 5, 'learning_rate': 0.08072125894416819, 'l2_leaf_reg': 2, 'bagging_temperature': 0.4357743674381209, 'min_data_in_leaf': 6}. Best is trial 0 with value: 0.7546504932949725.
[I 2024-11-22 04:28:20,264] Trial 1 finished with value: 0.7551300735770436 and parameters: {'iterations': 648, 'depth': 5, 'learning_rate': 0.08842520027292483, 'l2_leaf_reg': 4, 'bagging_temperature': 0.5708555909696973, 'min_data_in_leaf': 20}. Best is trial 0 with value: 0.7546504932949725.
[I 2024-11-22 04:28:33,928] Trial 2 finished with value: 0.7557534650492026 and parameters: {'iterations': 700, 'depth': 5, 'learning_rate': 0.04869023031649661, 'l2_leaf_reg': 10, 'bagging_temperature': 0.6241218428165092, 'min_data_in_leaf': 16}. Best is trial 0 with value: 0.7546504932949725

Best trial:
  Value: 0.7539628680195655
  Params: 
    iterations: 254
    depth: 5
    learning_rate: 0.08377009991199288
    l2_leaf_reg: 1
    bagging_temperature: 0.7204457890870082
    min_data_in_leaf: 4


In [31]:
import time
print(time.ctime())

Fri Nov 22 05:13:39 2024
